In [61]:
import pandas as pd

In [62]:
drugs_df = pd.read_csv('de.csv')

In [63]:
drugs_df[:3]

,DATE,STATUS,QLANG,DEM_GENDER,DEM_AGE,DEM_LOCATION,DEM_POSTAL,DEM_STDNT,DEM_VET,DEM_HEALTH,...,COD_NMU_NTY,DIHY_NMU_NTY,FENT_NMU_NTY,HYDM_NMU_NTY,METH_NMU_NTY,MORPH_NMU_NTY,OXY_NMU_NTY,SUF_NMU_NTY,TAP_NMU_NTY,TRAM_NMU_NTY
0,2018-01-12 08:08:01,3,2,1,21,1,786,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2018-01-14 08:08:30,3,2,1,24,1,797,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2018-01-14 07:31:08,3,2,1,19,1,717,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
def get_use_cat(df):
    # make new df so old dataset is not affected
    new_df = drugs_df.copy()
    # get list of drug names
    drugs = [x for x in df.columns if 'NMU' in x][:14]
    drugs = [x.split('_')[0] for x in drugs]
    print(drugs)
    # make categorical variable for each drug
    for drug in drugs:
        new_df[f'{drug}_USE_CAT'] = new_df[f'{drug}_USE'] + new_df[f'{drug}_NMU'].fillna(0)
        new_df[f'{drug}_USE_CAT'] = new_df[f'{drug}_USE_CAT'].astype(int)
    # in the returned df, each drug now has a column indicating how the correspondent uses the drug
    # 0   -> never used
    # 1   -> used for perscription purposes
    # 2   -> used for recreational purposes
    return new_df

In [65]:
drugs_df = get_use_cat(drugs_df)

['FENT', 'BUP', 'METH', 'MORPH', 'OXY', 'TRAM', 'TAP', 'SUF', 'COD', 'DIHY', 'HYDM', 'STIM', 'BENZ', 'THC']


In [78]:
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [72]:
y_variable = 'DAST_CAT'
X_variables = ['MENT_ANX', 'MENT_ADHD', 'MENT_AUT', 'MENT_BIP', 'MENT_BPD', 'MENT_DEP', 'MENT_EAT', 'MENT_OCD', 
               'MENT_PANIC', 'MENT_PPD', 'MENT_PTSD', 'MENT_SCH', 'MENT_OTH', 'MENT_NONE', 'TRAM_USE_CAT', 'OXY_USE_CAT', 'FENT_USE_CAT', 'BUP_USE_CAT', 'METH_USE_CAT', 'MORPH_USE_CAT',
       'TAP_USE_CAT', 'SUF_USE_CAT', 'COD_USE_CAT', 'DIHY_USE_CAT', 'HYDM_USE_CAT', 'STIM_USE_CAT',
       'BENZ_USE_CAT']

In [73]:
y = drugs_df[y_variable]
X = drugs_df[X_variables]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)

In [74]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [75]:
# Accuracy of the Training set
X_train_predict = clf.predict(X_train)
sum(y_train == X_train_predict)/len(X_train)

0.624031007751938

In [76]:
# Accuracy of the Testing set
X_test_predict = clf.predict(X_test)
sum(y_test == X_test_predict)/len(X_test)

0.519348945357914

In [79]:
# Neural Network
clf_nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 3), random_state=1)
clf_nn.fit(X_train, y_train)

/Users/yitongwu/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 3), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [80]:
# Accuracy of the Training set
X_train_predict_nn = clf_nn.predict(X_train)
sum(y_train == X_train_predict_nn)/len(X_train)

0.5532668881506091

In [81]:
# Accuracy of the Testing set
X_test_predict_nn = clf_nn.predict(X_test)
sum(y_test == X_test_predict_nn)/len(X_test)

0.5301444942700548